In [ ]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
!pip install pandasql
import pandas as pd

import numpy as np # linear algebra
import pandasql as psql

#FAST API
# !pip install python-multipart
%pip install colabcode fastapi jinja2 aiofiles
from fastapi import FastAPI, Request, Form
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates


#COLABCODE IMP
from colabcode import ColabCode
cc = ColabCode(port=12000, code=False)
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')


from typing import Optional

from fastapi import FastAPI
from pydantic import BaseModel

import json

import sys
sys.path.insert(0,'/content/drive/MyDrive/JobDescriptionPrediction/Project/utils')

import task2_insightsusingsql
from task2_insightsusingsql import *
import task8_modeloutputgenerator
from task8_modeloutputgenerator import *
import task9_evaluatingmodel
from task9_evaluatingmodel import *
from Task1_indeedJobScraping import fetch_jobs


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! cp -r drive/My\ Drive/JobDescriptionPrediction/Resources/Templates .
! cp drive/My\ Drive/JobDescriptionPrediction/Resources/Model/* .
!tar -xvf  'checkpoint_run1.tar' -C .
!tar -xvf  'checkpoint_run2.tar' -C .
!tar -xvf  'checkpoint_run3.tar' -C .


cp: -r not specified; omitting directory 'drive/My Drive/JobDescriptionPrediction/Resources/Model/T5-Aboutcompany'
cp: -r not specified; omitting directory 'drive/My Drive/JobDescriptionPrediction/Resources/Model/T5-Role'
cp: -r not specified; omitting directory 'drive/My Drive/JobDescriptionPrediction/Resources/Model/T5-Skill'
checkpoint/run1/
checkpoint/run1/checkpoint
checkpoint/run1/counter
checkpoint/run1/encoder.json
checkpoint/run1/events.out.tfevents.1617255936.c51996cb59be
checkpoint/run1/hparams.json
checkpoint/run1/model-3337.data-00000-of-00001
checkpoint/run1/model-3337.index
checkpoint/run1/model-3337.meta
checkpoint/run1/vocab.bpe
checkpoint/run2/
checkpoint/run2/checkpoint
checkpoint/run2/counter
checkpoint/run2/encoder.json
checkpoint/run2/events.out.tfevents.1617285150.0c4597a381ce
checkpoint/run2/hparams.json
checkpoint/run2/model-1000.data-00000-of-00001
checkpoint/run2/model-1000.index
checkpoint/run2/model-1000.meta
checkpoint/run2/vocab.bpe
checkpoint/run3/
check

In [ ]:
class Item(BaseModel):
    job_title: str
templates = Jinja2Templates(directory='Templates/')
app = FastAPI()
app.mount("/Templates", StaticFiles(directory="Templates"), name="Templates")

In [ ]:

#On index.html, two buttons we will have on index.html. On each button click, a new html page will be loaded then and there only.
#No api call required. Need home button on every page.
@app.get("/", response_class=HTMLResponse)
async def read_root(request: Request):
  return templates.TemplateResponse('index.html', context={'request':request})

@app.get("/home", response_class=HTMLResponse)
async def read_root(request: Request):
  return templates.TemplateResponse('index.html', context={'request':request})


@app.get("/textgenerationpage", response_class=HTMLResponse)
async def read_root(request: Request):
  return templates.TemplateResponse('textGeneration.html', context={'request':request})

  
@app.get("/datainsightsPage", response_class=HTMLResponse)
async def read_root(request: Request):
  return templates.TemplateResponse('datainsights.html', context={'request':request})


#Method to return PredictedJobDescription based on the inputs provided

@app.get("/predictJobDescription") 
async def predictJobDescription(request: Request, Input_Job_title : str,Input_Skills:str,Input_Company_name:str,Input_Location:str):
  jobDescription = getModelOutputSingleInput(Input_Job_title,Input_Skills,Input_Company_name,Input_Location)
  jobdescription = jobDescription.replace('\n','<br>')
  jobDescription = '<html><head><title>Predicted Job Description</title></head><body>'+'<h1>Predicted Job Description</h1><p>'+jobDescription+'</p></body></html>' 
  
  return jobDescription

@app.get("/evaluateModel", response_class=HTMLResponse) 
async def evaluateModel(request: Request, index1 : str ,index2 :str):
  df_row1 = getSimilarityMatrix(int(index1)) # this is a list output, needs to be added as  a row to df
  df_row2 = getSimilarityMatrix(int(index2))
  df_rows = [df_row1,df_row2]
  df = pd.DataFrame(df_rows, columns = ['Job_Title', 'Skills','Company','Location','Expected Output', 'Model Output','Word_similarity', '2Gram Similarity','4Gram Similarity','6Gram Similarity','8Gram Similarity','10Gram Similarity','Random No',])
  # add df_rows to df here
  df_html = df.to_html(classes='table table-striped')
  text = '<html><head><title>Cosine Similarity</title></head><body>'+'<h1>Similarity Matrix</h1>'+df_html+'</body></html>' 
  return text

#PAGE 2 - DataInsights.html 
@app.get("/getDataInsights", response_class=HTMLResponse)
async def getDataInsights(request: Request, id : str):
  print('got input :: ',id)
  if ( id == '1'):
    table = getSqlMethod1()
  elif ( id == '2'):
    table = getSqlMethod2()  
  elif ( id == '3'):
    table = getSqlMethod3()  
  elif ( id == '4'):
    table = getSqlMethod4() 
  elif ( id == '5'):
    table = getSqlMethod5() 
  elif ( id == '6'):
    table = getSqlMethod6()  
  elif ( id == '7'):
    table = getSqlMethod7()
  elif ( id == '8'):
    table = getSqlMethod8() 
  elif ( id == '9'):
    table = getSqlMethod9() 
  else:
    return '<html><head><title>Data Insights</title></head><body>'+'<h1>WRONG INPUT</h1>'+'</body></html>'   
    
  table = table.to_html(classes='table table-striped')
  text = '<html><head><title>Data Insights</title></head><body>'+ table+'</body></html>'  
  return text

@app.post("/datscrapping")
async def details(item: Item):
    item_dict = item.dict()
    data = fetch_jobs(item_dict["job_title"])
    data = json.loads(data)
    return {"result": data}


In [ ]:
cc.run_app(app=app)

Public URL: NgrokTunnel: "http://6413abc96d92.ngrok.io" -> "http://localhost:12000"


INFO:     Started server process [58]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [58]
